# Основы SciPy

SciPy - это библиотека для научных вычислений, включающая статистику, оптимизацию, интерполяцию и многое другое. Она критически важна для анализа биологических данных.

In [113]:
# Импорт библиотек
import numpy as np
import pandas as pd
import scipy
from scipy import stats
from scipy import optimize
from scipy import interpolate
from scipy import integrate

# Проверка версии
print(f"SciPy версия: {scipy.__version__}")

SciPy версия: 1.17.0


## 1. Статистические тесты (scipy.stats)

Самый важный модуль для биологов - статистический анализ данных.

### 1.1 t-test (сравнение средних)

In [114]:
# Данные экспрессии гена в двух группах
control = np.array([23.5, 25.1, 24.8, 26.2, 23.9, 25.5, 24.3, 25.8])
treated = np.array([28.3, 29.1, 27.5, 30.2, 28.7, 29.5, 28.1, 29.8])

print("Контроль:", control)
print("Обработка:", treated)
print(f"\nСреднее контроль: {np.mean(control):.2f}")
print(f"Среднее обработка: {np.mean(treated):.2f}")

Контроль: [23.5 25.1 24.8 26.2 23.9 25.5 24.3 25.8]
Обработка: [28.3 29.1 27.5 30.2 28.7 29.5 28.1 29.8]

Среднее контроль: 24.89
Среднее обработка: 28.90


In [115]:
# Независимый t-test (two-sample)
t_stat, p_value = stats.ttest_ind(control, treated)

print(f"\nt-статистика: {t_stat:.4f}")
print(f"p-value: {p_value:.6f}")

if p_value < 0.05:
    print("\nРезультат: Различия статистически значимы (p < 0.05)")
else:
    print("\nРезультат: Различия не значимы (p >= 0.05)")


t-статистика: -8.6219
p-value: 0.000001

Результат: Различия статистически значимы (p < 0.05)


In [116]:
# Парный t-test (paired samples)
# Например, измерения до и после лечения у одних и тех же пациентов
before = np.array([120, 135, 128, 142, 118, 130, 125, 138])
after = np.array([115, 128, 120, 135, 112, 125, 118, 130])

t_stat_paired, p_value_paired = stats.ttest_rel(before, after)

print("До лечения:", before)
print("После лечения:", after)
print(f"\nt-статистика: {t_stat_paired:.4f}")
print(f"p-value: {p_value_paired:.6f}")

До лечения: [120 135 128 142 118 130 125 138]
После лечения: [115 128 120 135 112 125 118 130]

t-статистика: 15.7765
p-value: 0.000001


### 1.2 Mann-Whitney U test (непараметрический)

In [117]:
# Когда данные не нормально распределены
group1 = np.array([12, 15, 18, 14, 16, 13, 17, 15])
group2 = np.array([22, 25, 21, 28, 24, 23, 26, 27])

u_stat, p_value_mw = stats.mannwhitneyu(group1, group2, alternative='two-sided')

print(f"U-статистика: {u_stat:.4f}")
print(f"p-value: {p_value_mw:.6f}")

if p_value_mw < 0.05:
    print("\nГруппы значимо различаются")
else:
    print("\nГруппы не различаются значимо")

U-статистика: 0.0000
p-value: 0.000931

Группы значимо различаются


### 1.3 ANOVA (сравнение нескольких групп)

In [118]:
# Экспрессия гена при трёх разных концентрациях препарата
dose_0 = np.array([100, 105, 98, 102, 101, 99])
dose_1 = np.array([110, 115, 108, 112, 111, 109])
dose_2 = np.array([125, 130, 122, 128, 127, 124])

# One-way ANOVA
f_stat, p_value_anova = stats.f_oneway(dose_0, dose_1, dose_2)

print("Дозировка 0:", dose_0)
print("Дозировка 1:", dose_1)
print("Дозировка 2:", dose_2)
print(f"\nF-статистика: {f_stat:.4f}")
print(f"p-value: {p_value_anova:.6f}")

if p_value_anova < 0.05:
    print("\nХотя бы одна группа значимо отличается от других")
else:
    print("\nВсе группы статистически неразличимы")

Дозировка 0: [100 105  98 102 101  99]
Дозировка 1: [110 115 108 112 111 109]
Дозировка 2: [125 130 122 128 127 124]

F-статистика: 139.3971
p-value: 0.000000

Хотя бы одна группа значимо отличается от других


### 1.4 Chi-square test (тест хи-квадрат)

In [119]:
# Таблица сопряжённости: тип лечения vs ответ
# Строки: Drug A, Drug B
# Столбцы: Response, No Response
contingency_table = np.array([
    [30, 10],  # Drug A: 30 ответили, 10 не ответили
    [15, 25]   # Drug B: 15 ответили, 25 не ответили
])

print("Таблица сопряжённости:")
print("           Response  No Response")
print(f"Drug A:      {contingency_table[0,0]}         {contingency_table[0,1]}")
print(f"Drug B:      {contingency_table[1,0]}         {contingency_table[1,1]}")

chi2, p_value_chi, dof, expected = stats.chi2_contingency(contingency_table)

print(f"\nХи-квадрат: {chi2:.4f}")
print(f"p-value: {p_value_chi:.6f}")
print(f"Степени свободы: {dof}")

if p_value_chi < 0.05:
    print("\nТип препарата связан с ответом на лечение")
else:
    print("\nНет связи между препаратом и ответом")

Таблица сопряжённости:
           Response  No Response
Drug A:      30         10
Drug B:      15         25

Хи-квадрат: 9.9556
p-value: 0.001604
Степени свободы: 1

Тип препарата связан с ответом на лечение


### 1.5 Корреляция

In [120]:
# Экспрессия двух генов
gene_a = np.array([120, 135, 128, 142, 118, 130, 125, 138, 122, 133])
gene_b = np.array([85, 95, 88, 98, 82, 90, 87, 96, 84, 92])

# Корреляция Пирсона (для линейной связи)
r_pearson, p_pearson = stats.pearsonr(gene_a, gene_b)

print(f"Корреляция Пирсона: r = {r_pearson:.4f}")
print(f"p-value: {p_pearson:.6f}")

# Корреляция Спирмена (для монотонной связи, непараметрический)
r_spearman, p_spearman = stats.spearmanr(gene_a, gene_b)

print(f"\nКорреляция Спирмена: rho = {r_spearman:.4f}")
print(f"p-value: {p_spearman:.6f}")

Корреляция Пирсона: r = 0.9885
p-value: 0.000000

Корреляция Спирмена: rho = 0.9879
p-value: 0.000000


### 1.6 Проверка на нормальность распределения

In [121]:
# Тест Шапиро-Уилка
np.random.seed(42)
normal_data = np.random.normal(100, 15, 30)
non_normal_data = np.random.exponential(2, 30)

# Для нормальных данных
stat_norm, p_norm = stats.shapiro(normal_data)
print("Нормальные данные:")
print(f"Статистика: {stat_norm:.4f}, p-value: {p_norm:.4f}")
if p_norm > 0.05:
    print("Данные согласуются с нормальным распределением")

# Для ненормальных данных
stat_non, p_non = stats.shapiro(non_normal_data)
print("\nНенормальные данные:")
print(f"Статистика: {stat_non:.4f}, p-value: {p_non:.4f}")
if p_non < 0.05:
    print("Данные не согласуются с нормальным распределением")

Нормальные данные:
Статистика: 0.9751, p-value: 0.6868
Данные согласуются с нормальным распределением

Ненормальные данные:
Статистика: 0.8443, p-value: 0.0005
Данные не согласуются с нормальным распределением


### 1.7 Множественное тестирование (коррекция p-value)

In [122]:
# Тестирование 100 генов
np.random.seed(42)
p_values = np.random.uniform(0, 0.1, 20)  # 20 p-values

print("Исходные p-values:")
print(p_values[:10])  # Первые 10

# Коррекция Бонферрони
bonferroni_threshold = 0.05 / len(p_values)
print(f"\nПорог Бонферрони: {bonferroni_threshold:.6f}")
significant_bonferroni = p_values < bonferroni_threshold
print(f"Значимых по Бонферрони: {np.sum(significant_bonferroni)}")

# Коррекция FDR (Benjamini-Hochberg)
from scipy.stats import false_discovery_control
rejected_fdr = false_discovery_control(p_values, method='bh')
print(f"Значимых по FDR (BH): {np.sum(rejected_fdr)}")

Исходные p-values:
[0.03745401 0.09507143 0.07319939 0.05986585 0.01560186 0.01559945
 0.00580836 0.08661761 0.0601115  0.07080726]

Порог Бонферрони: 0.002500
Значимых по Бонферрони: 1
Значимых по FDR (BH): 1.518341206949896


## 2. Регрессия и подбор кривых (curve fitting)

### 2.1 Линейная регрессия

In [123]:
# Калибровочная кривая (концентрация vs оптическая плотность)
concentrations = np.array([0, 10, 20, 30, 40, 50])  # мкг/мл
od_values = np.array([0.05, 0.23, 0.41, 0.59, 0.78, 0.95])

# Линейная регрессия
slope, intercept, r_value, p_value, std_err = stats.linregress(concentrations, od_values)

print(f"Уравнение: OD = {slope:.4f} * C + {intercept:.4f}")
print(f"R² = {r_value**2:.4f}")
print(f"p-value: {p_value:.6f}")
print(f"Стандартная ошибка: {std_err:.6f}")

Уравнение: OD = 0.0181 * C + 0.0495
R² = 0.9999
p-value: 0.000000
Стандартная ошибка: 0.000100


In [124]:
# Использование модели для предсказания
unknown_od = 0.65
predicted_concentration = (unknown_od - intercept) / slope

print(f"\nДля OD = {unknown_od}:")
print(f"Предсказанная концентрация: {predicted_concentration:.2f} мкг/мл")


Для OD = 0.65:
Предсказанная концентрация: 33.20 мкг/мл


### 2.2 Нелинейная регрессия (curve_fit)

In [125]:
# Кривая доза-ответ (4-параметрическая логистическая модель)
# Используется для IC50, EC50 и т.д.

# Данные
doses = np.array([0.1, 1, 10, 100, 1000, 10000])  # нМ
responses = np.array([5, 15, 45, 75, 90, 95])  # % ингибирования

print("Дозы (нМ):", doses)
print("Ответы (%):", responses)

Дозы (нМ): [1.e-01 1.e+00 1.e+01 1.e+02 1.e+03 1.e+04]
Ответы (%): [ 5 15 45 75 90 95]


In [126]:
# 4-параметрическая логистическая функция
def logistic_4pl(x, bottom, top, ic50, hill_slope):
    return bottom + (top - bottom) / (1 + (x / ic50) ** hill_slope)

# Начальные приближения параметров
initial_guess = [0, 100, 100, 1]

# Подбор параметров
params, covariance = optimize.curve_fit(
    logistic_4pl, 
    doses, 
    responses, 
    p0=initial_guess,
    maxfev=10000
)

bottom, top, ic50, hill_slope = params

print(f"\nПодобранные параметры:")
print(f"Bottom: {bottom:.2f}%")
print(f"Top: {top:.2f}%")
print(f"IC50: {ic50:.2f} нМ")
print(f"Hill slope: {hill_slope:.2f}")

/var/folders/gf/m55n227d05v7skfcl44wnd6c0000gn/T/ipykernel_49862/4081416617.py:3: RuntimeWarning: invalid value encountered in power
  return bottom + (top - bottom) / (1 + (x / ic50) ** hill_slope)


In [127]:
# Предсказание для новых доз
test_doses = np.array([5, 50, 500])
predicted_responses = logistic_4pl(test_doses, bottom, top, ic50, hill_slope)

print("\nПредсказания:")
for dose, resp in zip(test_doses, predicted_responses):
    print(f"Доза {dose} нМ: {resp:.1f}% ингибирование")


Предсказания:
Доза 5 нМ: 34.1% ингибирование
Доза 50 нМ: 67.5% ингибирование
Доза 500 нМ: 87.4% ингибирование


### 2.3 Экспоненциальная модель роста

In [128]:
# Рост бактериальной культуры
time_hours = np.array([0, 1, 2, 3, 4, 5, 6])
od600 = np.array([0.05, 0.08, 0.13, 0.21, 0.35, 0.58, 0.95])

# Экспоненциальная функция: y = a * exp(b * x)
def exponential_growth(t, a, b):
    return a * np.exp(b * t)

# Подбор параметров
params_exp, _ = optimize.curve_fit(exponential_growth, time_hours, od600)
a, b = params_exp

print(f"Уравнение роста: OD = {a:.4f} * exp({b:.4f} * t)")
print(f"\nВремя удвоения: {np.log(2) / b:.2f} часов")
print(f"Скорость роста (μ): {b:.4f} ч⁻¹")

Уравнение роста: OD = 0.0478 * exp(0.4983 * t)

Время удвоения: 1.39 часов
Скорость роста (μ): 0.4983 ч⁻¹


## 3. Интерполяция данных

### 3.1 Одномерная интерполяция

In [129]:
# Стандартная кривая
standard_conc = np.array([0, 25, 50, 100, 200, 400])  # нг/мл
standard_signal = np.array([0.05, 0.15, 0.28, 0.52, 0.95, 1.75])

print("Стандартная кривая:")
for conc, signal in zip(standard_conc, standard_signal):
    print(f"  {conc} нг/мл -> {signal} OD")

Стандартная кривая:
  0 нг/мл -> 0.05 OD
  25 нг/мл -> 0.15 OD
  50 нг/мл -> 0.28 OD
  100 нг/мл -> 0.52 OD
  200 нг/мл -> 0.95 OD
  400 нг/мл -> 1.75 OD


In [130]:
# Создание интерполятора
# Линейная интерполяция
f_linear = interpolate.interp1d(standard_signal, standard_conc, kind='linear')

# Кубическая интерполяция (более гладкая)
f_cubic = interpolate.interp1d(standard_signal, standard_conc, kind='cubic')

# Предсказание концентраций для неизвестных образцов
unknown_signals = np.array([0.20, 0.40, 0.80, 1.20])

print("\nПредсказание концентраций:")
print("Signal\tЛинейная\tКубическая")
for signal in unknown_signals:
    conc_lin = f_linear(signal)
    conc_cub = f_cubic(signal)
    print(f"{signal}\t{conc_lin:.1f}\t\t{conc_cub:.1f}")


Предсказание концентраций:
Signal	Линейная	Кубическая
0.2	34.6		35.0
0.4	75.0		74.0
0.8	165.1		164.1
1.2	262.5		261.4


### 3.2 Сплайн-интерполяция

In [131]:
# Для более сложных кривых
x_data = np.array([0, 2, 4, 6, 8, 10])
y_data = np.array([1.0, 2.5, 3.8, 3.2, 2.1, 1.5])

# Кубический сплайн
cs = interpolate.CubicSpline(x_data, y_data)

# Интерполяция в промежуточных точках
x_new = np.array([1, 3, 5, 7, 9])
y_new = cs(x_new)

print("Интерполированные значения:")
for x, y in zip(x_new, y_new):
    print(f"x = {x}: y = {y:.2f}")

Интерполированные значения:
x = 1: y = 1.62
x = 3: y = 3.33
x = 5: y = 3.68
x = 7: y = 2.63
x = 9: y = 1.69


## 4. Численное интегрирование

### 4.1 Интегрирование функций

In [132]:
# Вычисление площади под кривой (AUC)
# Например, фармакокинетика - концентрация препарата во времени

def drug_concentration(t):
    """Концентрация препарата в крови (мкг/мл) как функция времени (ч)"""
    return 100 * np.exp(-0.2 * t)

# Вычислить AUC от 0 до 24 часов
auc, error = integrate.quad(drug_concentration, 0, 24)

print(f"AUC (0-24 ч): {auc:.2f} мкг*ч/мл")
print(f"Погрешность: {error:.2e}")

AUC (0-24 ч): 495.89 мкг*ч/мл
Погрешность: 5.51e-12


### 4.2 Интегрирование табличных данных (трапеции)

In [133]:
# Экспериментальные данные концентрации
time_points = np.array([0, 1, 2, 4, 6, 8, 12, 24])  # часы
concentrations = np.array([0, 85, 95, 78, 62, 48, 28, 8])  # мкг/мл

print("Время (ч):", time_points)
print("Концентрация (мкг/мл):", concentrations)

# Метод трапеций
auc_trapz = integrate.trapezoid(concentrations, time_points)

print(f"\nAUC (метод трапеций): {auc_trapz:.2f} мкг*ч/мл")

Время (ч): [ 0  1  2  4  6  8 12 24]
Концентрация (мкг/мл): [ 0 85 95 78 62 48 28  8]

AUC (метод трапеций): 923.50 мкг*ч/мл


## 5. Распределения вероятностей

### 5.1 Нормальное распределение

In [134]:
# Параметры: среднее = 100, std = 15
mean = 100
std = 15

# Плотность вероятности в точке
x = 115
pdf_value = stats.norm.pdf(x, loc=mean, scale=std)
print(f"PDF в точке {x}: {pdf_value:.6f}")

# Кумулятивная вероятность (CDF) - вероятность что значение <= x
cdf_value = stats.norm.cdf(x, loc=mean, scale=std)
print(f"Вероятность значения <= {x}: {cdf_value:.4f} ({cdf_value*100:.1f}%)")

# Обратная функция - найти значение для заданной вероятности
percentile_95 = stats.norm.ppf(0.95, loc=mean, scale=std)
print(f"\n95-й перцентиль: {percentile_95:.2f}")

PDF в точке 115: 0.016131
Вероятность значения <= 115: 0.8413 (84.1%)

95-й перцентиль: 124.67


In [135]:
# Генерация случайных чисел из распределения
samples = stats.norm.rvs(loc=mean, scale=std, size=1000, random_state=42)

print(f"\nГенерация 1000 значений:")
print(f"Среднее выборки: {np.mean(samples):.2f}")
print(f"Std выборки: {np.std(samples):.2f}")


Генерация 1000 значений:
Среднее выборки: 100.29
Std выборки: 14.68


### 5.2 Биномиальное распределение

In [136]:
# Пример: вероятность мутаций
# n = количество испытаний (клеток), p = вероятность мутации
n_cells = 100
p_mutation = 0.05

# Вероятность ровно k мутаций
k = 5
prob_exact = stats.binom.pmf(k, n_cells, p_mutation)
print(f"Вероятность ровно {k} мутаций из {n_cells} клеток: {prob_exact:.4f}")

# Вероятность <= 3 мутаций
prob_leq_3 = stats.binom.cdf(3, n_cells, p_mutation)
print(f"Вероятность <= 3 мутаций: {prob_leq_3:.4f}")

# Вероятность >= 8 мутаций
prob_geq_8 = 1 - stats.binom.cdf(7, n_cells, p_mutation)
print(f"Вероятность >= 8 мутаций: {prob_geq_8:.4f}")

Вероятность ровно 5 мутаций из 100 клеток: 0.1800
Вероятность <= 3 мутаций: 0.2578
Вероятность >= 8 мутаций: 0.1280


### 5.3 Пуассоновское распределение

In [137]:
# Используется для подсчёта редких событий
# Например, количество мутаций в геноме
lambda_param = 3.5  # среднее количество событий

# Вероятность ровно k событий
for k in range(0, 8):
    prob = stats.poisson.pmf(k, lambda_param)
    print(f"P(X = {k}) = {prob:.4f}")

P(X = 0) = 0.0302
P(X = 1) = 0.1057
P(X = 2) = 0.1850
P(X = 3) = 0.2158
P(X = 4) = 0.1888
P(X = 5) = 0.1322
P(X = 6) = 0.0771
P(X = 7) = 0.0385


## 6. Практические примеры из биологии

### 6.1 Анализ дифференциальной экспрессии генов

In [138]:
# Данные экспрессии 10 генов в двух условиях (по 3 реплики)
np.random.seed(42)

n_genes = 10
gene_names = [f'Gene_{i+1}' for i in range(n_genes)]

# Контроль и обработка
control_replicates = np.random.negative_binomial(20, 0.3, (n_genes, 3))
treated_replicates = control_replicates * np.random.uniform(0.5, 2.5, (n_genes, 1))
treated_replicates = treated_replicates.astype(int)

# Создание DataFrame
df = pd.DataFrame({
    'Gene': gene_names,
    'Control_1': control_replicates[:, 0],
    'Control_2': control_replicates[:, 1],
    'Control_3': control_replicates[:, 2],
    'Treated_1': treated_replicates[:, 0],
    'Treated_2': treated_replicates[:, 1],
    'Treated_3': treated_replicates[:, 2]
})

print("Данные экспрессии:")
print(df)

Данные экспрессии:
      Gene  Control_1  Control_2  Control_3  Treated_1  Treated_2  Treated_3
0   Gene_1         53         46         34         79         68         50
1   Gene_2         50         34         42         55         37         46
2   Gene_3         33         43         40         35         45         42
3   Gene_4         62         58         20         35         33         11
4   Gene_5         40         38         42         68         65         72
5   Gene_6         46         40         39         69         60         58
6   Gene_7         54         68         44         32         41         26
7   Gene_8         60         49         45         63         51         47
8   Gene_9         57         45         50        132        104        115
9  Gene_10         31         39         54         30         38         52


In [139]:
# Статистический анализ для каждого гена
results = []

for idx, row in df.iterrows():
    gene = row['Gene']
    control = row[['Control_1', 'Control_2', 'Control_3']].values.astype(float)
    treated = row[['Treated_1', 'Treated_2', 'Treated_3']].values.astype(float)
    
    # t-test
    t_stat, p_val = stats.ttest_ind(control, treated)
    
    # Fold change
    fc = np.mean(treated) / (np.mean(control) + 1)
    log2_fc = np.log2(fc)
    
    results.append({
        'Gene': gene,
        'Mean_Control': np.mean(control),
        'Mean_Treated': np.mean(treated),
        'Fold_Change': fc,
        'Log2_FC': log2_fc,
        'p_value': p_val,
        't_statistic': t_stat
    })

results_df = pd.DataFrame(results)

print("\nРезультаты анализа:")
print(results_df.round(4))


Результаты анализа:
      Gene  Mean_Control  Mean_Treated  Fold_Change  Log2_FC  p_value  \
0   Gene_1       44.3333       65.6667       1.4485   0.5346   0.1025   
1   Gene_2       42.0000       46.0000       1.0698   0.0973   0.5959   
2   Gene_3       38.6667       40.6667       1.0252   0.0359   0.6580   
3   Gene_4       46.6667       26.3333       0.5524  -0.8561   0.2581   
4   Gene_5       40.0000       68.3333       1.6667   0.7370   0.0003   
5   Gene_6       41.6667       62.3333       1.4609   0.5469   0.0068   
6   Gene_7       55.3333       33.0000       0.5858  -0.7715   0.0530   
7   Gene_8       51.3333       53.6667       1.0255   0.0363   0.7406   
8   Gene_9       50.6667      117.0000       2.2645   1.1792   0.0017   
9  Gene_10       41.3333       40.0000       0.9449  -0.0818   0.8931   

   t_statistic  
0      -2.1100  
1      -0.5754  
2      -0.4773  
3       1.3174  
4     -12.1429  
5      -5.1312  
6       2.7194  
7      -0.3549  
8      -7.4895  
9    

In [140]:
# Коррекция на множественное тестирование
results_df['p_adjusted_BH'] = false_discovery_control(
    results_df['p_value'].values, method='bh'
).astype(float)

# Значимо дифференциально экспрессированные гены
results_df['Significant'] = (results_df['p_adjusted_BH'] < 0.05) & (np.abs(results_df['Log2_FC']) > 1)

print("\nС коррекцией FDR:")
print(results_df[['Gene', 'Log2_FC', 'p_value', 'p_adjusted_BH', 'Significant']].round(4))

print(f"\nКоличество значимых генов: {results_df['Significant'].sum()}")


С коррекцией FDR:
      Gene  Log2_FC  p_value  p_adjusted_BH  Significant
0   Gene_1   0.5346   0.1025         0.2050        False
1   Gene_2   0.0973   0.5959         0.8225        False
2   Gene_3   0.0359   0.6580         0.8225        False
3   Gene_4  -0.8561   0.2581         0.4302        False
4   Gene_5   0.7370   0.0003         0.0026        False
5   Gene_6   0.5469   0.0068         0.0228        False
6   Gene_7  -0.7715   0.0530         0.1325        False
7   Gene_8   0.0363   0.7406         0.8229        False
8   Gene_9   1.1792   0.0017         0.0085         True
9  Gene_10  -0.0818   0.8931         0.8931        False

Количество значимых генов: 1


### 6.2 Расчёт IC50 для препарата

In [141]:
# Данные доза-ответ (жизнеспособность клеток)
concentrations = np.array([0.01, 0.1, 1, 10, 100, 1000])  # мкМ
viability = np.array([98.5, 96.2, 85.3, 52.1, 15.3, 5.2])  # %

print("Концентрация (мкМ):", concentrations)
print("Жизнеспособность (%):", viability)

# Модель: 4-параметрический логистический
def dose_response_4pl(x, bottom, top, ic50, slope):
    return bottom + (top - bottom) / (1 + (x / ic50) ** slope)

# Подбор параметров
initial_params = [0, 100, 10, 1]
params, cov = optimize.curve_fit(
    dose_response_4pl,
    concentrations,
    viability,
    p0=initial_params,
    bounds=([0, 80, 0.001, -5], [20, 110, 10000, 5])
)

bottom, top, ic50, slope = params

print(f"\nПараметры модели:")
print(f"Bottom (минимум): {bottom:.2f}%")
print(f"Top (максимум): {top:.2f}%")
print(f"IC50: {ic50:.3f} мкМ")
print(f"Hill slope: {slope:.2f}")

# Доверительный интервал для IC50
perr = np.sqrt(np.diag(cov))
ic50_stderr = perr[2]
print(f"\nIC50 ± SE: {ic50:.3f} ± {ic50_stderr:.3f} мкМ")

Концентрация (мкМ): [1.e-02 1.e-01 1.e+00 1.e+01 1.e+02 1.e+03]
Жизнеспособность (%): [98.5 96.2 85.3 52.1 15.3  5.2]

Параметры модели:
Bottom (минимум): 2.33%
Top (максимум): 98.50%
IC50: 10.597 мкМ
Hill slope: 0.80

IC50 ± SE: 10.597 ± 0.598 мкМ


### 6.3 Анализ выживаемости (Kaplan-Meier like)

In [142]:
# Время выживания пациентов (месяцы)
group_a_survival = np.array([12, 15, 18, 20, 22, 25, 28, 30, 35, 40])
group_b_survival = np.array([8, 10, 12, 14, 15, 16, 18, 20, 22, 25])

print("Группа A (месяцы):", group_a_survival)
print("Группа B (месяцы):", group_b_survival)

print(f"\nМедиана выживаемости A: {np.median(group_a_survival):.1f} мес")
print(f"Медиана выживаемости B: {np.median(group_b_survival):.1f} мес")

# Сравнение групп (Mann-Whitney)
u_stat, p_val = stats.mannwhitneyu(group_a_survival, group_b_survival, alternative='two-sided')
print(f"\nMann-Whitney U test:")
print(f"U-статистика: {u_stat:.2f}")
print(f"p-value: {p_val:.4f}")

if p_val < 0.05:
    print("Группы значимо различаются")
else:
    print("Различия не значимы")

Группа A (месяцы): [12 15 18 20 22 25 28 30 35 40]
Группа B (месяцы): [ 8 10 12 14 15 16 18 20 22 25]

Медиана выживаемости A: 23.5 мес
Медиана выживаемости B: 15.5 мес

Mann-Whitney U test:
U-статистика: 79.00
p-value: 0.0308
Группы значимо различаются


### 6.4 Анализ ферментативной кинетики (Михаэлиса-Ментен)

In [143]:
# Данные: концентрация субстрата vs скорость реакции
substrate_conc = np.array([1, 2, 5, 10, 20, 50, 100, 200])  # мкМ
reaction_rate = np.array([0.5, 0.9, 1.8, 2.5, 3.2, 4.0, 4.3, 4.5])  # мкмоль/мин

print("[Субстрат] (мкМ):", substrate_conc)
print("Скорость (мкмоль/мин):", reaction_rate)

[Субстрат] (мкМ): [  1   2   5  10  20  50 100 200]
Скорость (мкмоль/мин): [0.5 0.9 1.8 2.5 3.2 4.  4.3 4.5]


In [144]:
# Уравнение Михаэлиса-Ментен: v = Vmax * [S] / (Km + [S])
def michaelis_menten(s, vmax, km):
    return vmax * s / (km + s)

# Подбор параметров
params_mm, cov_mm = optimize.curve_fit(
    michaelis_menten,
    substrate_conc,
    reaction_rate,
    p0=[5, 10]
)

vmax, km = params_mm

print(f"\nПараметры фермента:")
print(f"Vmax: {vmax:.2f} мкмоль/мин")
print(f"Km: {km:.2f} мкМ")
print(f"\nKm = концентрация субстрата при половине максимальной скорости")


Параметры фермента:
Vmax: 4.66 мкмоль/мин
Km: 8.48 мкМ

Km = концентрация субстрата при половине максимальной скорости


In [145]:
# Расчёт kcat (если известна концентрация фермента)
enzyme_conc = 0.1  # мкМ
kcat = vmax / enzyme_conc

print(f"\nkcat (число оборотов): {kcat:.1f} с⁻¹")
print(f"Каталитическая эффективность (kcat/Km): {kcat/km:.2f} мкМ⁻¹с⁻¹")


kcat (число оборотов): 46.6 с⁻¹
Каталитическая эффективность (kcat/Km): 5.50 мкМ⁻¹с⁻¹


### 6.5 Стандартизация qPCR данных

In [146]:
# qPCR данные: Ct values
qpcr_data = pd.DataFrame({
    'Sample': ['Control_1', 'Control_2', 'Control_3', 'Treated_1', 'Treated_2', 'Treated_3'],
    'Target_Gene': [22.5, 22.8, 22.3, 20.1, 20.4, 19.9],
    'Reference_Gene': [18.2, 18.5, 18.3, 18.1, 18.4, 18.2]
})

print("qPCR данные:")
print(qpcr_data)

# Расчёт ΔCt
qpcr_data['Delta_Ct'] = qpcr_data['Target_Gene'] - qpcr_data['Reference_Gene']

# Средний ΔCt для контроля
control_delta_ct = qpcr_data[qpcr_data['Sample'].str.contains('Control')]['Delta_Ct'].mean()

# Расчёт ΔΔCt
qpcr_data['Delta_Delta_Ct'] = qpcr_data['Delta_Ct'] - control_delta_ct

# Fold change: 2^(-ΔΔCt)
qpcr_data['Fold_Change'] = 2 ** (-qpcr_data['Delta_Delta_Ct'])

print("\nС расчётами:")
print(qpcr_data.round(3))

qPCR данные:
      Sample  Target_Gene  Reference_Gene
0  Control_1         22.5            18.2
1  Control_2         22.8            18.5
2  Control_3         22.3            18.3
3  Treated_1         20.1            18.1
4  Treated_2         20.4            18.4
5  Treated_3         19.9            18.2

С расчётами:
      Sample  Target_Gene  Reference_Gene  Delta_Ct  Delta_Delta_Ct  \
0  Control_1         22.5            18.2       4.3             0.1   
1  Control_2         22.8            18.5       4.3             0.1   
2  Control_3         22.3            18.3       4.0            -0.2   
3  Treated_1         20.1            18.1       2.0            -2.2   
4  Treated_2         20.4            18.4       2.0            -2.2   
5  Treated_3         19.9            18.2       1.7            -2.5   

   Fold_Change  
0        0.933  
1        0.933  
2        1.149  
3        4.595  
4        4.595  
5        5.657  


In [147]:
# Статистическое сравнение
control_fc = qpcr_data[qpcr_data['Sample'].str.contains('Control')]['Fold_Change'].values
treated_fc = qpcr_data[qpcr_data['Sample'].str.contains('Treated')]['Fold_Change'].values

t_stat, p_val = stats.ttest_ind(control_fc, treated_fc)

print(f"\nСреднее fold change:")
print(f"Контроль: {np.mean(control_fc):.2f} ± {np.std(control_fc):.2f}")
print(f"Обработка: {np.mean(treated_fc):.2f} ± {np.std(treated_fc):.2f}")
print(f"\nt-test: t = {t_stat:.3f}, p = {p_val:.4f}")


Среднее fold change:
Контроль: 1.00 ± 0.10
Обработка: 4.95 ± 0.50

t-test: t = -10.917, p = 0.0004


## 7. Практические задания

### Задание 1: Сравнение трёх методов лечения

In [148]:
# Данные: уровень маркера после лечения
method_a = np.array([85, 88, 82, 90, 87, 84, 89, 86])
method_b = np.array([92, 95, 90, 98, 93, 91, 96, 94])
method_c = np.array([78, 82, 76, 84, 80, 79, 83, 81])

# Задание:
# 1. Проверьте, есть ли значимые различия между группами (ANOVA)
# 2. Если различия есть, проведите попарные сравнения (t-tests)
# 3. Примените коррекцию Бонферрони для множественных сравнений

# Ваш код здесь

### Задание 2: Калибровочная кривая ELISA

In [149]:
# Стандарты с известной концентрацией
std_conc = np.array([0, 10, 20, 50, 100, 200, 500])  # пг/мл
std_od = np.array([0.08, 0.15, 0.22, 0.45, 0.78, 1.35, 2.15])

# Неизвестные образцы
sample_od = np.array([0.35, 0.62, 0.95, 1.58])

# Задание:
# 1. Постройте калибровочную кривую (используйте 4PL или логарифмическую регрессию)
# 2. Определите концентрации в неизвестных образцах
# 3. Оцените качество подгонки (R²)

# Ваш код здесь

### Задание 3: Анализ роста бактериальной культуры

In [150]:
# Данные роста (OD600 во времени)
time_h = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8])
od600 = np.array([0.05, 0.07, 0.12, 0.20, 0.35, 0.58, 0.85, 1.15, 1.38])

# Задание:
# 1. Подберите экспоненциальную модель роста
# 2. Рассчитайте скорость роста (μ) и время удвоения
# 3. Предскажите OD600 через 10 часов

# Ваш код здесь